In [1]:
import sys
import numpy as np 
import xarray as xr 
import matplotlib.pyplot as plt 

In [2]:
src_fp = "glc1-a/nc/glc1-a_dx_50_NT_60_dt_0.5_MB_-1.225_OFF_Tma_-9.00_B_0.003_pseudo_gridded.nc"

with xr.open_dataset(src_fp) as src_ds: 
    pass

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html
http://xarray.pydata.org/en/stable/user-guide/io.html

In [ ]:
fig, ax = plt.subplots(figsize=(6,3), constrained_layout=True)

im = ax.pcolormesh(src_ds.X[:,::-1]/1e3,
                   src_ds.Z.isel(t=-1), 
                   src_ds.temperature.isel(t=-1), 
                   shading='gouraud')

ax.plot(src_ds.X.isel(coord_2=-1)[::-1]/1e3, 
        src_ds.Z.isel(coord_2=-1, t=0),
        color='k', alpha=0.5, lw=1.0)

cbar = fig.colorbar(im)
cbar.set_label('Homologous Temperature [$^\circ$ C]', 
               rotation=270, labelpad=15)

ax.set_ylabel('Elevation [m a.s.l.]')
ax.set_xlabel('Distance from headwall [km]')

fig.savefig('/Users/andrewnolan/Desktop/abstract_draft.jpeg', 
            bbox_inches='tight', facecolor='w', dpi=600)

In [ ]:
fig, ax = plt.subplots(1,1)

im = ax.pcolormesh(src_ds.X.isel(coord_2=-1), 
                   src_ds.t,
                   src_ds.height.isel(coord_2=-1).differentiate('t'),
                   shading='gouraud',
                   clim=(-15,15), vmin=-15, vmax=15, cmap='RdBu')

mask = (src_ds.enthalpy_h <= src_ds['phase change enthalpy']).isel(coord_2=0)

ax.pcolor(src_ds.X.isel(coord_2=-1), 
          src_ds.t, 
          src_ds.enthalpy_h.isel(coord_2=-1).where(mask), 
          alpha=0.0,
          hatch='///', 
          
          zorder=2)

plt.colorbar(im, extend='both')

plt.savefig('dHdt.pdf', bbox_inches='tight', facecolor='w')

In [ ]:
mask = (src_ds.enthalpy_h >= src_ds['phase change enthalpy']).isel(coord_2=0)

XX, TT = np.meshgrid(mask.X, mask.t)

In [ ]:
src_ds.enthalpy_h.isel(coord_2=-1).where(mask)

In [ ]:
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

def detrend(da, dims, deg=1):
    # detrend along multiple dimensions
    # only valid for linear detrending (deg=1)
    da_detrended = da
    for dim in dims:
        da_detrended = detrend_dim(da_detrended, dim, deg=deg)
    return da_detrended

In [ ]:
detrend(src_ds.surface_enthalpy.isel(coord_2=-1), ['coord_1']).plot()

In [ ]:
src_ds.surface_enthalpy.dims